In [1]:
import pandas as pd
import numpy as np


In [2]:
def splitTrainTest(data,percent):
    total=len(data)
    trainTotal=int(total*percent*0.01)
    testTotal=total-trainTotal
    return (data[0:trainTotal],data[trainTotal:total])

In [108]:
class Layer:
    def __init__(self,nNodesCurrent, nNodesNext, activationF):
        self.nodesNo=nNodesCurrent
        self.activations = np.zeros([nNodesCurrent,1])
        self.activationF=activationF
    
        if nNodesNext==0:
            self.weights=None
        else:
            self.weights=np.random.normal(0, 1, size=(nNodesCurrent,nNodesNext))

In [171]:
class NeuralNet:
    def __init__(self, totalLayers, noNodesList, activationFunctions):
        self.totalLayers=totalLayers
        self.noNodesList=noNodesList
        self.layers = []
        for i in range(totalLayers):
            currentLayerNodes=noNodesList[i]
            if i!=totalLayers-1:
                nextLayerNodes=noNodesList[i+1]
                ith_Layer=Layer(currentLayerNodes,nextLayerNodes,activationFunctions[i])
            else:
                ith_Layer=Layer(currentLayerNodes,0,activationFunctions[i])
            self.layers.append(ith_Layer)#append output layer as none

    def trainNetwork(self, data,outputLabels, batchSize, epochs, learningRate):
        self.learningRate=learningRate
        self.batchSize=batchSize;
        
        #normalize data
#         data=((data-data.min(axis=0))/(data.max(axis=0)-data.min(axis=0)))
        data=data/255
#         data=((data-data.mean(axis=0))/(data.std(axis=0)))
        
        for x in range(epochs):
            i=0  
            while i<len(data):
                self.error=0
                self.forwardPropo(data[i:i+batchSize])#input
                self.calculateError(outputLabels[i:i+batchSize])#output
#                 print("==============Batch",i,"================")
                self.back_pass(outputLabels[i:i+batchSize])
                i+=batchSize
            self.error /= batchSize
            print("Error: ",x, self.error)
          
    def forwardPropo(self, inputs):
        self.layers[0].activations =inputs
        for i in range(self.totalLayers-1):
#             print(self.layers[i].activations.shape,self.layers[i].weights.shape,)
            temp=np.matmul(self.layers[i].activations,self.layers[i].weights)  
           # print(temp)
#             print("==============================")
            if self.layers[i+1].activationF == "sigmoid":
                self.layers[i+1].activations = self.sigmoid(temp)
            elif self.layers[i+1].activationF == "softmax":
                self.layers[i+1].activations = self.softmax(temp)
            elif self.layers[i+1].activationF == "relu":
                self.layers[i+1].activations = self.relu(temp)
            elif self.layers[i+1].activationF == "tanh":
                self.layers[i+1].activations = self.tanh(temp)
            else:
                self.layers[i+1].activations = temp
            # print(self.layers[self.totalLayers-1].activations)
        
    def calculateError(self,labels):
#         print(labels.shape)
        if len(labels[0]) != self.layers[self.totalLayers-1].nodesNo:
            print ("Error: Label is not of the same shape as output layer.")
            print("Label: ", len(labels), " : ", len(labels[0]))
            print("Out: ", len(self.layers[self.totalLayers-1].activations), " : ", len(self.layers[self.totalLayers-1].activations[0]))
            return
        self.error += np.negative(np.sum(np.multiply(labels, np.log(self.layers[self.totalLayers-1].activations))))
    
    def back_pass(self, labels):
        # if self.cost_function == "cross_entropy" and self.layers[self.num_layers-1].activation_function == "softmax":
        targets = labels
        i = self.totalLayers-1
        y = self.layers[i].activations
        
        delta=(y-targets)
        deltaw = np.dot(self.layers[i-1].activations.T, delta)/self.batchSize
        new_weights = self.layers[i-1].weights - self.learningRate * deltaw
        for i in range(i-1, 0, -1):
            
            sigPrime = self.sigmoidDerivative(np.matmul(self.layers[i-1].activations,self.layers[i-1].weights))
#             reluPrime = self.relu_derivative(np.matmul(self.layers[i-1].activations,self.layers[i-1].weights))            
#             tanPrime = self.tanhDerivative(np.matmul(self.layers[i-1].activations,self.layers[i-1].weights))
            
            delta=np.multiply(sigPrime,delta.dot(self.layers[i].weights.T))
            deltaw = np.dot(self.layers[i-1].activations.T, delta)/self.batchSize

            self.layers[i].weights = new_weights
            new_weights = self.layers[i-1].weights - self.learningRate * deltaw
        self.layers[0].weights = new_weights
            
    def check_accuracy(self, inputs, labels):
        self.batchSize = len(inputs)
        self.forwardPropo(inputs)
        a = self.layers[self.totalLayers-1].activations
        print(len(a))
        total=0
        correct=0
        for i in range(len(a)):
            total += 1
            al = a[i].tolist()
#             print(al,labels[i])
            if labels[i][al.index(max(al))] == 1:
                correct += 1
        print(correct)
        print("Accuracy: ", correct*100/total)
    
    def sigmoid(self, x):
        return np.divide(1, np.add(1, np.exp(np.negative(x))))
    
    def sigmoidDerivative(self,x):
        return (self.sigmoid(x)*(1-self.sigmoid(x)))
    
    def relu(self, x):
        return x * (x > 0)#/700
    
    def relu_derivative(self,X):
        return 1. * (X > 0)
    
    def softmax(self, x):
        exp = np.exp(x)
        if isinstance(x[0], np.ndarray):
            return exp/np.sum(exp, axis=1, keepdims=True)
        else:
            return exp/np.sum(exp, keepdims=True)

    def tanh(self, x):
        return np.tanh(x)
    
    def tanhDerivative(self,x):
        return 1.0 - np.tanh(x) ** 2

In [42]:
data=pd.read_csv("../input/apparel-trainval.csv").values

In [172]:
def getOneHotLabels(data,k):
    one_hot_labels = np.zeros((len(data), k))
    for i in range(len(data)):  
        one_hot_labels[i,data[i,0]] = 1
    return one_hot_labels

In [175]:
train,test=splitTrainTest(data,80)
oneHotLabelsTrain=getOneHotLabels(train,10)
oneHotLabelsTest=getOneHotLabels(test,10)
print(train.shape)
trainInputs=train[:,1:]
testInputs=test[:,1:]
print(trainInputs.shape)
nn=NeuralNet(4,[784,16,16,10],[None,"sigmoid","sigmoid","softmax"])
# nn=NeuralNet(4,[784,16,16,10],[None,"tanh","tanh","softmax"])
# nn=NeuralNet(4,[784,16,16,10],[None,"relu","relu","softmax"])

(48000, 785)
(48000, 784)


In [176]:
# nn.trainNetwork(trainInputs,oneHotLabelsTrain,100,50,0.01)
nn.trainNetwork(trainInputs,oneHotLabelsTrain,64,50,0.1)

nn.check_accuracy( testInputs, oneHotLabelsTest)

Error:  0 1.1917104599122805
Error:  1 0.9055784486982013
Error:  2 0.80134140524004
Error:  3 0.7489179120202868
Error:  4 0.7029784567273103
Error:  5 0.6710474267856397
Error:  6 0.6483171086153241
Error:  7 0.6284838142368887
Error:  8 0.6179170255694653
Error:  9 0.6119074205054404
Error:  10 0.6008123783402415
Error:  11 0.5888943205678343
Error:  12 0.5790301170541488
Error:  13 0.5713148801360531
Error:  14 0.5632796151370592
Error:  15 0.5509785713830453
Error:  16 0.5379543405043368
Error:  17 0.5271498804779837
Error:  18 0.5182222824422325
Error:  19 0.5103022085837647
Error:  20 0.502810092264328
Error:  21 0.49526153126726113
Error:  22 0.48754903935839244
Error:  23 0.47978963058680274
Error:  24 0.47191456454749925
Error:  25 0.46363806053645873
Error:  26 0.45475255304626977
Error:  27 0.445323631090673
Error:  28 0.4355312555048888
Error:  29 0.42556295640453917
Error:  30 0.41562102657330924
Error:  31 0.4059050886468479
Error:  32 0.3965645877675605
Error:  33 0.387

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:103: RuntimeWarning: overflow encountered in exp
